In [ ]:
from gtda.homology import EuclideanCechPersistence,VietorisRipsPersistence
from gtda.diagrams import PersistenceImage
from gtda.pipeline import Pipeline
from gtda.diagrams import Filtering

In [ ]:
import tensorflow as tf
import gudhi as gd
import numpy as np
import matplotlib.pyplot as plt
import utils
import scipy.spatial as spatial
import json
from rich import print
from random import choice, sample
from tqdm.notebook import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors
from gc import collect

with open("data/drugbank.json", mode = "r") as drugbank:
    db = json.load(drugbank)

%matplotlib inline

In [ ]:
print(dir(Descriptors))

In [ ]:
print(Descriptors.MolLogP(Chem.MolFromSmiles("Cc1cc(CCCOc2c(C)cc(cc2C)c3noc(n3)C(F)(F)F)on1")))

# Transform data to point clouds

In [ ]:
spectra = dict()

mol_indices = list(sample(list(range(len(db))), 2000))
mol_strings = np.asarray(list(db.keys()))[ mol_indices ]

for m in tqdm(mol_strings):
    energies = [
        np.asarray([ utils.split_fpn(x) for x in db[m]["energy0"]["mz"] ]),
        np.asarray([ utils.split_fpn(x) for x in db[m]["energy1"]["mz"] ]),
        np.asarray([ utils.split_fpn(x) for x in db[m]["energy2"]["mz"] ])
    ]
    for i,energy in enumerate(energies):
        energies[i][:,0] = np.log(energy[:,0])/np.log(6000)
        
    spectra[m] = tuple(energies)

mol_strings = list(spectra.keys())
del mol_indices
del db
collect()

# Inspect data

In [ ]:
fig = plt.figure(figsize=(17,17))
ax = fig.add_subplot()
for mol_s in sample(mol_strings, min(len(mol_strings), 100) ):
    (X0,X1,X2) = spectra[ mol_s ]
    ax.scatter(X0[:,0],X0[:,1], color = "blue")
    ax.scatter(X1[:,0],X1[:,1], color = "green")
    ax.scatter(X2[:,0],X2[:,1], color = "red")

# Crossval split

In [ ]:
train_mols = sample(mol_strings, int(np.ceil(len(mol_strings)*0.8)) )
test_mols = list(set(mol_strings) - set(train_mols))

train_spectra = { m : spectra[m] for m in train_mols }
test_spectra = { m : spectra[m] for m in test_mols }

assert set(train_spectra.keys()).intersection(set(test_spectra.keys())) == set()
print('# train_mols: ',len(train_mols), '\t # test mols: ', len(test_mols))

# Compute Čech persistence

In [ ]:
def compute_cech_persistence(spectra):
    pipeline = Pipeline([
        ('diagram', EuclideanCechPersistence(
            homology_dimensions = [0,1,2],
            max_edge_length = np.sqrt(2),
            n_jobs = 3,
        )),
        ('filtering', Filtering()),
        ('image', PersistenceImage(n_bins = 20, n_jobs = 3)),
    ])
    # dictionary of 3-tuples of point clouds
    molecules = list(spectra.keys())
    x0 = [ X[0] for X in spectra.values() ]
    x1 = [ X[1] for X in spectra.values() ]
    x2 = [ X[2] for X in spectra.values() ]
    data = (
        pipeline.fit_transform(x0),
        pipeline.fit_transform(x1),
        pipeline.fit_transform(x2),
    )
    data = np.concatenate(data, axis = 1)
    return molecules, data

train_molecules, train_data = compute_cech_persistence(train_spectra)
test_molecules, test_data = compute_cech_persistence(test_spectra)

In [ ]:
print(train_data.shape)

In [ ]:
plt.imshow(train_data[16,2])

# Extract Labels
(this can be done from rdkit or from the descriptor .csv)

In [ ]:
my_descriptors = (
    "NumAromaticRings",
    "ExactMolWt",
    "NumValenceElectrons",
    "MolLogP"
)# interesting, but domain difficulties

integer_descriptors = (
    'NumAliphaticCarbocycles',
    'NumAliphaticHeterocycles',
    'NumAliphaticRings',
    'NumAromaticCarbocycles',
    'NumAromaticHeterocycles',
    'NumAromaticRings',
    'NumHAcceptors',
    'NumHDonors',
    'NumHeteroatoms',
    'NumRadicalElectrons',
    'NumRotatableBonds',
    'NumSaturatedCarbocycles',
    'NumSaturatedHeterocycles',
    'NumSaturatedRings',
#    'NumValenceElectrons'
)
logpvsa_descriptors = (
    'SlogP_VSA1',
    'SlogP_VSA10',
    'SlogP_VSA11',
    'SlogP_VSA12',
    'SlogP_VSA2',
    'SlogP_VSA3',
    'SlogP_VSA4',
    'SlogP_VSA5',
    'SlogP_VSA6',
    'SlogP_VSA7',
    'SlogP_VSA8',
    'SlogP_VSA9'
)
#descriptors = integer_descriptors
descriptors = logpvsa_descriptors

num_descriptors = len(descriptors)

train_labels = np.zeros((train_data.shape[0], num_descriptors))
test_labels = np.zeros((test_data.shape[0], num_descriptors))

for i,d in enumerate(descriptors):
    get_desc = getattr(Descriptors, d)
    train_labels[:,i] = np.asarray(list(map(
        lambda mol : get_desc(Chem.MolFromSmiles(mol)),
        train_molecules
    )))
    test_labels[:,i] = np.asarray(list(map(
        lambda mol : get_desc(Chem.MolFromSmiles(mol)),
        test_molecules
    )))

In [ ]:
print(choice(train_labels))

# Setup CNN

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        filters = 32,
        kernel_size = 3, # sliding window
        input_shape = train_data.shape[1:],
        data_format = "channels_first",
        #padding = "same",
        activation = "relu",
    ),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(
        filters = 100,
        kernel_size = 3,
        #data_format = "channels_first",
        #padding = "same",
        activation = "relu",
    ),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64),#, activation = "relu"),
    #tf.keras.layers.Discretization(bin_boundaries=list(range(100000))),
    tf.keras.layers.Dense(num_descriptors),
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = "mean_absolute_error",    
)

In [ ]:
%%time
history = model.fit(
    train_data,
    train_labels,
    verbose = 1,
    epochs = 100,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2
)

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    #plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)
plot_loss(history)

In [ ]:
compare = np.zeros((test_data.shape[0], num_descriptors, 2))
compare[:,:,0] = test_labels
compare[:,:,1] = model.predict(test_data).reshape((test_data.shape[0],num_descriptors))

with np.printoptions(precision = 2, suppress = True):
    for t in compare:
        print(t)